# Test LightGBM with Bayesian Optimsation

# Feature Engineering

In [12]:
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error,r2_score
import time

import lightgbm as lgb
from lightgbm import LGBMRegressor
from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings("ignore")

In [13]:
### Thanks to Shinji Suzuki

# OS
import glob, re
from datetime import datetime
import pickle

# data science tool
import numpy as np
import pandas as pd
import datetime as dt


# machine learning
from sklearn import *
from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb

# データの読み込み
# 事前にcalendar_dateをvisit_dataに変更しています。airとhpgで同じことですが、別名で使用されているようです。
data = {
    'tra': pd.read_csv('../../../mltestdata/05_recruit/air_visit_data.csv'),
    'as': pd.read_csv('../../../mltestdata/05_recruit/air_store_info.csv'),
    'hs': pd.read_csv('../../../mltestdata/05_recruit/hpg_store_info.csv'),
    'ar': pd.read_csv('../../../mltestdata/05_recruit/air_reserve.csv'),
    'hr': pd.read_csv('../../../mltestdata/05_recruit/hpg_reserve.csv'),
    'id': pd.read_csv('../../../mltestdata/05_recruit/store_id_relation.csv'),
    'tes': pd.read_csv('../../../mltestdata/05_recruit/sample_submission.csv'),
    'hol': pd.read_csv('../../../mltestdata/05_recruit/date_info.csv').rename(columns={'calendar_date':'visit_date'})
}

# それぞれのデータをマージするために、まずは、relation用のものをマージします
data['hr'] = pd.merge(data['hr'], data['id'], how = 'inner', on = ['hpg_store_id'])

for df in ['ar', 'hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r:(r['visit_datetime']- r['reserve_datetime']).days, axis = 1)
    tmp1 = data[df].groupby(['air_store_id', 'visit_datetime'], as_index =False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns = {'visit_datetime':'visit_date', 'reserve_datetime_diff':'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id', 'visit_datetime'], as_index =False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns = {'visit_datetime':'visit_date', 'reserve_datetime_diff':'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how = 'inner', on = ['air_store_id', 'visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()

stores = pd.concat([pd.DataFrame({'air_store_id':unique_stores, 'dow':[i]*len(unique_stores)}) for i in range(7)], axis =0, ignore_index = True).reset_index(drop = True) 

#曜日だけでなく、月も追加
stores_m = pd.concat([pd.DataFrame({'air_store_id':unique_stores, 'month':[i]*len(unique_stores)}) for i in range(1,13)], axis =0, ignore_index = True).reset_index(drop = True)
stores = pd.merge(stores_m, stores,on=('air_store_id'), how='left')

tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].min().rename(columns = {'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].median().rename(columns = {'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].max().rename(columns = {'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].count().rename(columns = {'visitors':'count_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])

#曜日だけでなく、ID×月も追加
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].min().rename(columns = {'visitors':'m_min_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'m_mean_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].median().rename(columns = {'visitors':'m_median_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].max().rename(columns = {'visitors':'m_max_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].count().rename(columns = {'visitors':'m_count_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])

stores = pd.merge(stores, data['as'], how= "left", on = ['air_store_id'])

stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/', ' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-', ' ')))

lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name' + str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
    stores['air_area_name' + str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

#土日フラグ(day_of_week_1)と、休日前(holi_2)フラグを追加
data['hol']['day_of_week_1']= data['hol']['day_of_week'].replace(['Saturday', 'Sunday','Monday','Tuesday','Wednesday','Thursday','Friday'],['1', '1','0','0','0','0','0']).astype('int')
data['hol']['holi_2'] = data['hol'][['holiday_flg', 'day_of_week_1']].sum(axis = 1)
data['hol']['holi_2'] = data['hol']['holi_2'].apply( lambda x: 0 if x < 1 else 1 )
data['hol']['holi_2'] = data['hol']['holi_2'].shift(-1)
data['hol']['holi_2'] = data['hol']['holi_2'].fillna(1)
data['hol']['holi_2'] = data['hol']['holi_2'].astype('int')

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how ='left', on = ['visit_date'])
test = pd.merge(data['tes'], data['hol'], how ='left', on = ['visit_date'])

#曜日と月でmerge
train = pd.merge(train, stores, how ='left', on = ['air_store_id', 'dow','month'])
test = pd.merge(test, stores, how ='left', on = ['air_store_id', 'dow','month'])

#ID×休日前でのvisitorsの平均、中央値等を追加
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].min().rename(columns = {'visitors':'h_min_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'h_mean_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].median().rename(columns = {'visitors':'h_median_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].max().rename(columns = {'visitors':'h_max_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].count().rename(columns = {'visitors':'h_count_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1) 

train['total_reserve_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserve_mean'] = (train['rv2_x'] + train['rv2_y'])/2
train['total_reserve_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y'])/2

test['total_reserve_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserve_mean'] = (test['rv2_x'] + test['rv2_y'])/2
test['total_reserve_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y'])/2

train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2']= lbl.fit_transform(test['air_store_id'])

train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date', 'visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat([train, test]) 

#指数移動平均の追加。これはなくても良いかも
#https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting/discussion/46179#266344
#def calc_shifted_ewm(series, alpha, adjust=True):
#    return series.shift().ewm(alpha=alpha, adjust=adjust).mean()

#train['ewm'] = train.groupby(['air_store_id', 'dow']).apply(lambda g: calc_shifted_ewm(g['visitors'], 0.1)).sort_index(level=['air_store_id']).values

#以下、気象データの追加
df_air_store_weather_station = pd.read_csv('../../../mltestdata/05_recruit/air_store_info_with_nearest_active_station.csv')

cols = ['air_store_id', 'station_id', 'station_latitude', 'station_longitude', 'station_vincenty', 'station_great_circle']
all_data = pd.merge(all_data, df_air_store_weather_station[cols], on='air_store_id', how='left')

combine = all_data
filenames = []
df_weather = None
for station_id in combine['station_id'].unique():
    fn = f"../../../mltestdata/05_recruit/1-1-16_5-31-17_Weather/{station_id}.csv"
    if not fn in filenames:
        df = pd.read_csv(fn)
        df['station_id'] = station_id
        if df_weather is None:
            df_weather = df
        else:
            df_weather = pd.concat([df_weather, df])
        del df

        filenames.append(fn)
    else:
        continue

#欠損値を平均で穴埋め（median, ffillで試すも特に差は出なかった）
df_weather = df_weather.fillna(df_weather.mean())

df_weather = df_weather.rename(columns={'calendar_date': 'visit_date'})

df_weather['visit_date'] = pd.to_datetime(df_weather['visit_date'])
df_weather['visit_date'] = df_weather['visit_date'].dt.date

#なんとなく対数化
df_weather['precipitation'] = np.log1p(df_weather['precipitation'])

#使いそうなデータだけ結合（その他の気象データは試していません。特に意味はなし）
cols = ['station_id', 
    'visit_date', 
    'precipitation', 
    'hours_sunlight',
    'avg_temperature',
    'high_temperature',
    'low_temperature']

combine = pd.merge(combine, df_weather[cols], on=['station_id', 'visit_date'], how='left')

#降水量をカテゴリ化
def simplify_pre(df):
    df.precipitation = df.precipitation.fillna(0)
    bins = ( -1, 0.01, 2,  5)
    group_names = ['1', '2', '3']
    categories = pd.cut(df.precipitation, bins, labels=group_names)
    df.precipitation = categories
    return df

combine = simplify_pre(combine) 
all_data = combine 

#不要そうなデータを削除
drop_col =['station_id', 'station_latitude','station_longitude','station_vincenty', 'station_great_circle','hours_sunlight','high_temperature','low_temperature']
all_data = all_data.drop(drop_col, axis = 1)

train = all_data[:ntrain]
test = all_data[ntrain:]

#ID×降水量で平均、中央値等を追加
tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].min().rename(columns = {'visitors':'p_min_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'p_mean_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].median().rename(columns = {'visitors':'p_median_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation',])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].max().rename(columns = {'visitors':'p_max_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].count().rename(columns = {'visitors':'p_count_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

#countをLabel Encoder化。なんとなく試してみたら結果が良かった。
lbl = preprocessing.LabelEncoder()
train['count_visitors'] = lbl.fit_transform(train['count_visitors']) 
test['count_visitors']= lbl.fit_transform(test['count_visitors'])
train['m_count_visitors'] = lbl.fit_transform(train['m_count_visitors'])
test['m_count_visitors']= lbl.fit_transform(test['m_count_visitors'])
train['h_count_visitors'] = lbl.fit_transform(train['h_count_visitors'])
test['h_count_visitors']= lbl.fit_transform(test['h_count_visitors'])
train['p_count_visitors'] = lbl.fit_transform(train['p_count_visitors'])
test['p_count_visitors']= lbl.fit_transform(test['p_count_visitors'])

# GW flag
combine = [train, test]
gw_list = ['2016-04-29','2016-04-30','2016-05-01','2016-05-02','2016-05-03','2016-05-04','2016-05-05','2017-04-29','2017-04-30','2017-05-01','2017-05-02','2017-05-03','2017-05-04','2017-05-05']
post_gw_list=['2016-05-06']
train['gw_flg'] = 0
train['post_gw_flg'] = 0
test['gw_flg'] = 0
test['post_gw_flg'] = 0
update_gw_list = [["0" for i in range(3)] for j in range(len(gw_list))]
update_post_gw_list = [["0" for i in range(3)] for j in range(len(post_gw_list))]

from datetime import date
for index, gw_date in enumerate(gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_gw_list[index][col_i]=int(temp_figure)
        
    #print("{}  {}  {}".format(update_list[index][0],update_list[index][1],update_list[index][2]))
    
for index, gw_date in enumerate(post_gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_post_gw_list[index][col_i]=int(temp_figure)

for dataset in combine:
    for index in range(len(update_gw_list)):
        dataset.loc[dataset.visit_date == date(update_gw_list[index][0],update_gw_list[index][1],update_gw_list[index][2]), 'gw_flg'] = 1
        
for dataset in combine:
    for index in range(len(update_post_gw_list)):
        dataset.loc[dataset.visit_date == date(update_post_gw_list[index][0],update_post_gw_list[index][1],update_post_gw_list[index][2]), 'post_gw_flg'] = 1     

In [14]:
train.columns

Index(['air_area_name', 'air_area_name0', 'air_area_name1', 'air_area_name2',
       'air_area_name3', 'air_area_name4', 'air_area_name5', 'air_area_name6',
       'air_area_name7', 'air_area_name8', 'air_area_name9', 'air_genre_name',
       'air_genre_name0', 'air_genre_name1', 'air_genre_name2',
       'air_genre_name3', 'air_genre_name4', 'air_genre_name5',
       'air_genre_name6', 'air_genre_name7', 'air_genre_name8',
       'air_genre_name9', 'air_store_id', 'air_store_id2', 'count_visitors',
       'date_int', 'day_of_week', 'day_of_week_1', 'dow', 'h_count_visitors',
       'h_max_visitors', 'h_mean_visitors', 'h_median_visitors',
       'h_min_visitors', 'holi_2', 'holiday_flg', 'id', 'latitude',
       'lon_plus_lat', 'longitude', 'm_count_visitors', 'm_max_visitors',
       'm_mean_visitors', 'm_median_visitors', 'm_min_visitors',
       'max_visitors', 'mean_visitors', 'median_visitors', 'min_visitors',
       'month', 'rs1_x', 'rs1_y', 'rs2_x', 'rs2_y', 'rv1_x', 'rv1_y', 

In [15]:
drop_cols=['visitors','air_store_id','visit_date','id']
y_train=train['visitors']
x_train=train.drop(drop_cols, axis=1)

x_test=test.copy()
x_test=x_test.drop(drop_cols, axis=1)

In [16]:
y = train.visitors
train_input = train.copy()
test_input = test.copy()

drop_cols=['visitors','air_store_id','visit_date','id']
train_input=train_input.drop(drop_cols, axis=1)
test_input=test_input.drop(drop_cols, axis=1)

# Modeling

### LightGBM

In [17]:
#Define a evaluation function

def rmsle(preds, true):
    rmsle = np.sqrt(mean_squared_error(np.log1p(true), np.log1p(preds)))
    return float(rmsle)

In [18]:
#Define a evaluation matrix 
from sklearn.metrics.scorer import make_scorer
RMSLE = make_scorer(rmsle)

In [19]:
# Define a function for comparing predictions and true data.
def compare_result(preds, true):
    compare = pd.DataFrame({"test_id": true.index,
                           "real_cost": true,
                           "pred_cost": preds})
    compare = compare[["test_id", "real_cost", "pred_cost"]].reset_index(drop=True)
    
    compare["error_percent_(%)"] = np.abs(compare.real_cost - compare.pred_cost) / compare.real_cost * 100
    
    return compare

# Cross validation with LightGBM

In [20]:
def cross_validate_lgb(params, x_train, y_train, x_test, 
                        kf, 
                        cat_features=[],
                        verbose=True, verbose_eval=100, nseeds=1, df_input=True,
                        early_stopping=100, num_boost_round=8000, scoreonly=False):

    start_time = time.time()
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))

    # self-defined eval metric
    # f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
    # binary error
    def feval_rmsle(preds, train_data):
        preds = np.expm1(preds)
        true = np.expm1(train_data.get_label())

        return 'rmsle', rmsle(preds, true), False
       
    if len(cat_features)==0: use_cat=False

    # use the k-fold object to enumerate indexes for each training and validation fold
    for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train)): # folds 1, 2 ,3 ,4, 5
        # example: training from 1,2,3,4; validation from 5
        if df_input:
            x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[val_index, :]
        else:
            x_train_kf, x_val_kf = x_train[train_index], x_train[val_index]

        y_train_kf, y_val_kf = np.log1p(y_train[train_index]), np.log1p(y_train[val_index])

        for seed in range(nseeds):
            params['feature_fraction_seed'] = seed
            params['bagging_seed'] = seed

            if use_cat:
                lgb_train = lgb.Dataset(x_train_kf, y_train_kf, categorical_feature=cat_features)
                lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train, categorical_feature=cat_features)

            else:
                lgb_train = lgb.Dataset(x_train_kf, y_train_kf)
                lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train)

            gbm = lgb.train(params,
                            lgb_train,
                            num_boost_round=num_boost_round,
                            valid_sets=[lgb_val],
                            early_stopping_rounds=early_stopping,
                            feval=feval_rmsle,
                            verbose_eval=verbose_eval)

            val_pred = np.expm1(gbm.predict(x_val_kf, num_iteration=gbm.best_iteration))
      
            train_pred[val_index] += val_pred
            test_pred += np.expm1((gbm.predict(x_test, num_iteration=gbm.best_iteration)))

        train_pred[val_index] = val_pred/nseeds

        #fold_rmsle = rmsle(np.expm1(y_val_kf.values), train_pred[val_index])
        fold_rmsle = rmsle(train_pred[val_index],np.expm1(y_val_kf.values))
        if verbose:
            print('fold cv {} RMSLE score is {:.6f}'.format(i, fold_rmsle))

    test_pred = test_pred / (nseeds * kf.n_splits)
    #cv_score = rmsle(y_train, train_pred)
    cv_score = rmsle(train_pred, y_train)
    
    if verbose:
        print('cv RMSLE score is {:.6f}'.format(cv_score))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
    #return cv_score, np.expm1(train_pred),test_pred
    
    if scoreonly:
        return cv_score
    else:
        return cv_score, train_pred, test_pred

In [21]:
# only do 5 fold CV here so that we save some running time on Kaggle Kernel
kf=StratifiedKFold(n_splits=10, shuffle=True, random_state=2018)
#kf=KFold(n_splits=3, shuffle=True, random_state=2018)

In [22]:
lgb_params = {
    'boosting_type': 'dart',
    'max_depth' : 5,
    'max_bin' : 500,
    'learning_rate': 0.1,  # 0.618580
    'num_leaves': 22,
    #'metric': 'RMSE'
}


print('Start training...')

#cv_score =cross_validate_lgb(lgb_params, train_input, y, kf, verbose=True, verbose_eval=50,df_input=True)
#cv_score =cross_validate_lgb_nofeval(lgb_params, train_input, y, kf, verbose=True, verbose_eval=50,df_input=True)
cv_score =cross_validate_lgb(lgb_params, train_input, y, test_input, kf, verbose=True, verbose_eval=50,df_input=True,scoreonly=True)

print('cv score is {:.6f}'.format(cv_score))

Start training...
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmsle: 0.576527
[100]	valid_0's rmsle: 0.580981
Early stopping, best iteration is:
[45]	valid_0's rmsle: 0.50369
fold cv 0 RMSLE score is 1.790145
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmsle: 0.567732
[100]	valid_0's rmsle: 0.572222
Early stopping, best iteration is:
[45]	valid_0's rmsle: 0.50179
fold cv 1 RMSLE score is 1.777082
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmsle: 0.565892
[100]	valid_0's rmsle: 0.57032
Early stopping, best iteration is:
[45]	valid_0's rmsle: 0.498261
fold cv 2 RMSLE score is 1.776979
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmsle: 0.567081
[100]	valid_0's rmsle: 0.571596
Early stopping, best iteration is:
[45]	valid_0's rmsle: 0.49811
fold cv 3 RMSLE score is 1.777877
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmsl

# Bayesian Optimsation - Setup

In [24]:
params={
    'num_leaves':(7,4095),#(7,4095),
    'max_depth':(2,63),
    'learning_rate':(0.05,0.3),
    'scale_pos_weight':(1,10000),
    'min_sum_hessian_in_leaf':(2,30),
    'subsample':(0.4,1.0),
    'colsample_bytree':(0.4,1.0),
    'feature_fraction':(0.1,0.9),
    'bagging_fraction':(0.1,0.9),
    'bagging_freq':(0,2),
    'lambda_l1':(0.0,1.0),
    'lambda_l2':(0.0,1.0),
    'n_estimators':(2,30), 
    'reg_lambda':(0.0,2.0),
    'min_gain_to_split':(0.0,1.0)
}

In [25]:
# reload(lgb_wrapper)
#def lgbcv_func(max_depth, learning_rate, subsample, colsample_bytree, nthread=4, seed=0):
def lgbcv_func(num_leaves, max_depth, learning_rate,
               scale_pos_weight, 
               min_sum_hessian_in_leaf, 
               subsample, 
               colsample_bytree,
               feature_fraction, bagging_fraction, 
               bagging_freq, lambda_l1, lambda_l2,
               n_estimators,reg_lambda,min_gain_to_split,
               nthread=4):

    params = {
        'objective' : "regression",
        'task': 'train',
        'boosting_type': 'dart',
                
        'num_leaves': int(num_leaves),
        'max_depth': int(max_depth), 
        'learning_rate': float(learning_rate),
        'scale_pos_weight':scale_pos_weight,
        'min_sum_hessian_in_leaf':float(min_sum_hessian_in_leaf), 
        'subsample':subsample,
        'colsample_bytree':colsample_bytree,
        'feature_fraction':feature_fraction, 
        'bagging_fraction':bagging_fraction,
        'bagging_freq':int(bagging_freq), 
        'lambda_l1':lambda_l1, 
        'lambda_l2':lambda_l2,
        'n_estimators':n_estimators,
        'reg_lambda':reg_lambda,
        'min_gain_to_split':min_gain_to_split       
        #'metric': 'RMSE'
    }
    
    # for a more ideal out-of-fold model prediction for this dataset, we use 10-fold CV
    kf=StratifiedKFold(n_splits=3, shuffle=True, random_state=2018)
    
    # we will disable all the verbose setting in this functional call, so that we don't have too much information 
    # to read during the bayesian optimisation process.
    return 1-cross_validate_lgb(params, train_input, y, test_input, kf, verbose=False, verbose_eval=False, scoreonly=True)

In [26]:
lgb_bo=BayesianOptimization(lgbcv_func, params)

In [27]:
lgb_bo.maximize(init_points=5, n_iter=15)

Initialization
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   bagging_fraction |   bagging_freq |   colsample_bytree |   feature_fraction |   lambda_l1 |   lambda_l2 |   learning_rate |   max_depth |   min_gain_to_split |   min_sum_hessian_in_leaf |   n_estimators |   num_leaves |   reg_lambda |   scale_pos_weight |   subsample | 
    1 | 05m25s |    0.48378 |             0.6544 |         0.2736 |             0.6421 |             0.6178 |      0.0028 |      0.7316 |          0.2562 |     31.8783 |              0.6564 |                    6.1172 |        16.1972 |    2154.0919 |       0.9616 |          9547.2113 |      0.5334 | 
    2 | 01m40s |   -0.71211 |             0.4542 |         1.1918 |          

In [28]:
print('-'*30)
print('Maximum value: %f' % lgb_bo.res['max']['max_val'])
print('Best parameters: ', lgb_bo.res['max']['max_params'])

------------------------------
Maximum value: 0.497862
Best parameters:  {'num_leaves': 242.70662305751688, 'max_depth': 48.434894524736237, 'learning_rate': 0.22739737221785333, 'scale_pos_weight': 4449.6905120668471, 'min_sum_hessian_in_leaf': 2.9535851736769945, 'subsample': 0.78319355511875155, 'colsample_bytree': 0.60980110433768919, 'feature_fraction': 0.38725837624747139, 'bagging_fraction': 0.25993005950069908, 'bagging_freq': 0.0063304070505429966, 'lambda_l1': 0.10468672203086582, 'lambda_l2': 0.73311141753276088, 'n_estimators': 7.574050255399829, 'reg_lambda': 0.7867690387142483, 'min_gain_to_split': 0.32711184887807354}


# Velification

#### Test #1

In [23]:
lgb_params = {
    'num_leaves': int(242.70662305751688), 
    'max_depth': int(48.434894524736237), 
    'learning_rate': 0.22739737221785333, 
    'scale_pos_weight': 4449.6905120668471, 
    'min_sum_hessian_in_leaf': 2.9535851736769945, 
    'subsample': 0.78319355511875155, 
    'colsample_bytree': 0.60980110433768919, 
    'feature_fraction': 0.38725837624747139, 
    'bagging_fraction': 0.25993005950069908, 
    'bagging_freq': int(0.0063304070505429966), 
    'lambda_l1': 0.10468672203086582, 
    'lambda_l2': 0.73311141753276088, 
    'n_estimators': 7.574050255399829, 
    'reg_lambda': 0.7867690387142483, 
    'min_gain_to_split': 0.32711184887807354} 

outcomes=cross_validate_lgb(lgb_params, train_input, y, test_input, kf, verbose_eval=False)

lgb_cv=outcomes[0]
lgb_train_pred=outcomes[1]
lgb_test_pred=outcomes[2]

lgb_train_pred_df=pd.DataFrame(columns=['visitors'], data=lgb_train_pred)
lgb_test_pred_df=pd.DataFrame(columns=['visitors'], data=lgb_test_pred)

fold cv 0 RMSLE score is 0.480030
fold cv 1 RMSLE score is 0.478435
fold cv 2 RMSLE score is 0.474693
fold cv 3 RMSLE score is 0.474436
fold cv 4 RMSLE score is 0.468857
fold cv 5 RMSLE score is 0.473968
fold cv 6 RMSLE score is 0.475132
fold cv 7 RMSLE score is 0.471494
fold cv 8 RMSLE score is 0.472199
fold cv 9 RMSLE score is 0.469553
cv RMSLE score is 0.473897
it takes 104.521 seconds to perform cross validation


# Saving data

In [33]:
lv1_lgb_train_pred = lgb_train_pred_df.copy()
lv1_lgb_train_pred.to_csv('lv1_lgb_train_pred.csv', index=False)

lv1_lgb_test_pred = lgb_test_pred_df.copy()
lv1_lgb_test_pred.to_csv('lv1_lgb_test_pred.csv', index=False)

# Submission

In [35]:
test['visitors'] = lgb_test_pred_df.values
sub = test[['id','visitors']].copy()
sub.to_csv('submission_rs_recruit_v15_lgbm_feature_engineering_shinji_01.csv', index=False)
print('Finished.')
# lgbm
# fe by shinji, ryo
# bopt
# LB: 0.484

Finished.


# Last work with weight

In [36]:
sub1 = sub.copy()

In [37]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('../../../mltestdata/05_recruit/*.csv')}

for k, v in dfs.items(): locals()[k] = v

wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub_merge = pd.merge(sub1, sub2, on='id', how='inner')

In [39]:
sub_merge['visitors'] = (sub_merge['visitors_x'] + sub_merge['visitors_y']* 1.1)/2
sub_merge[['id', 'visitors']].to_csv('submission_rs_recruit_v15_lgbm_feature_engineering_shinji_02.csv', index=False)
# lgbm
# fe by shinji, ryo
# weight
# LB: 0.481

* * * * *
# Appendix.

# Blending

In [41]:
sub_automl = pd.read_csv('../../../mltestdata/05_recruit/submission_automl.csv')
sub_lgbv15 = pd.read_csv('./submission_rs_recruit_v15_lgbm_feature_engineering_02.csv')
sub_blending = pd.DataFrame()
sub_blending['id'] = sub_lgbboptweight['id']

In [47]:
rate_automl = 0.8
rate_lgbboptweight = 0.2

In [ ]:
sub_blending['visitors'] = rate_automl*sub_automl['visitors']+rate_lgbboptweight*sub_lgbboptweight['visitors']
sub_blending.to_csv('submission_rs_recruit_v11_lgbm_v04_03.csv', index=False)
# LB 0.480